In [112]:
# Import all libraries
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.preprocessing import StandardScaler

In [113]:
# Read in dataframe
df = pd.read_csv('fbdh1.csv')

In [ ]:
# df.head()

In [114]:
# Use a label fix function to assign classifications or ground truth to Flow thresholds
def label_fix(label):
    if label < -10:
        return 0
    elif label > 10:
        return 1
    else:
        return 2

df['Class'] = df['Flow'].apply(label_fix)

In [115]:
df.head()

,Flow,D1,D2,P1,P2,dD1,dD2,dP1,dP2,Class
0,-16.69,99.49,-145.71,48.04,-15.90,81.17,12.26,36.71,2.21,0
1,-16.23,172.78,-118.66,79.84,-11.94,80.09,18.49,35.48,2.91,0
2,-15.54,242.66,-81.28,109.77,-7.75,75.74,24.65,33.03,3.26,0
3,-15.17,316.42,-55.50,140.84,-5.30,74.07,28.43,31.89,3.37,0
4,-14.51,392.40,-31.67,171.92,-1.24,73.96,29.70,31.32,3.68,0


In [116]:
# Create the Feature Matrix and Scale Features
X = df.drop('Flow', axis = 1)
X.drop('Class', axis = 1, inplace = True)

scaler = StandardScaler()
scaler.fit(X)
scaled_features = scaler.transform(X)
X = pd.DataFrame(scaled_features, columns = X.columns[:])

# Create the classification matrix
y = df['Class']

# Perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

columns = ['D1', 'D2', 'P1', 'P2', 'dD1', 'dD2', 'dP1', 'dP2']

In [117]:
features = [tf.feature_column.numeric_column(key = column) for column in columns]
print(features[0])

NumericColumn(key='D1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)


In [118]:
# Create training input function
training_input_fn = tf.estimator.inputs.pandas_input_fn(x = X_train,
                                                        y = y_train,
                                                        batch_size = 50,
                                                        shuffle= True,
                                                        num_epochs = None)
# Create testing input function
eval_input_fn = tf.estimator.inputs.pandas_input_fn(x = X_test,
                                                    y = y_test,
                                                    batch_size = 50,
                                                    shuffle = False,
                                                    num_epochs = 1)

In [119]:
# Create Linear Estimator
LinearClass = tf.estimator.LinearClassifier(feature_columns = features, n_classes = 3, model_dir = 'LinearModel')

In [120]:
# Train Linear Estimator
LinearClass.train(input_fn = training_input_fn, steps = 5000)

In [121]:
LinearClass.evaluate(input_fn = eval_input_fn)

{'accuracy': 0.8585,
 'average_loss': 0.3584422,
 'loss': 17.92211,
 'global_step': 10000}

In [123]:
# Read in new testing dataframe
test = pd.read_csv('test_data_2.csv')
test['Class'] = test['Flow'].apply(label_fix)

In [ ]:
# Convert to TensorFlow Lite

In [104]:
def serving_input_receiver_fn():
  feat_dict = {'D1': tf.placeholder(tf.float32),
                'D2': tf.placeholder(tf.float32), 
                'P1': tf.placeholder(tf.float32), 
                'P2': tf.placeholder(tf.float32),
                'dD1': tf.placeholder(tf.float32), 
                'dD2': tf.placeholder(tf.float32), 
                'dP1': tf.placeholder(tf.float32), 
                'dP2': tf.placeholder(tf.float32)}
  return tf.estimator.export.ServingInputReceiver(
    feat_dict, 
    feat_dict)

In [107]:
model_dir = 'Linear_Model'
LinearClass.export_saved_model(model_dir, serving_input_receiver_fn)

W0905 13:28:07.604056 4413126080 export_utils.py:182] Export includes no default signature!


b'Linear_Model/1567704487'